In [33]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.metrics import MeanAbsolutePercentageError
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


import time
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input

from funcionesComunes import *


In [34]:
# Cargar el archivo CSV con punto y coma como delimitador
df = pd.read_csv('RCPMerged/Abies spectabilis_merged.csv')
df = df[~df["nametag"].str.startswith("INDI005")]

# Codificamos, normalización y split de datos
df = codification(df)
print(df.shape)

df, valorNormalizacion = individualNormalization(df)
print(f"SE HA NORMALIZADO EL ARCHIVO: Abies spectabilis_merged")

temp_df = df
train_data, val_data, test_data = split_population_individuals(temp_df, train_pct=0.80, val_pct_in_train=0.20, details=False)
train_data.shape, val_data.shape, test_data.shape

# Obtenemos X e y para los datasets de train, val y test 
WINDOWS_SIZE = 3
X_train, y_train = df_to_X_y_ind_3(train_data, WINDOWS_SIZE)
X_val, y_val = df_to_X_y_ind_3(val_data, WINDOWS_SIZE)
X_test, y_test = df_to_X_y_ind_3(test_data, WINDOWS_SIZE)
print(X_train.shape, X_test.shape, X_val.shape)


(96429, 77)
SE HA NORMALIZADO EL ARCHIVO: Abies spectabilis_merged
(56618, 4, 43) (19637, 4, 43) (15689, 4, 43)


In [132]:
modelLSTM = tf.keras.models.load_model("models/modelLSTM.keras")
modelLSTMFreeze = modelLSTM

modelLSTM.summary()

/home/workstation/Escritorio/PabloC/TransferLearning/.venv/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 18 variables whereas the saved optimizer has 34 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 4, 64)          │        27,648 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 4, 128)         │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 4, 128)         │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 4, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 24)             │         1,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 24)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            25 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 618,084 (2.36 MB)

 Trainable params: 309,041 (1.18 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 309,043 (1.18 MB)

# Congelación de capas

In [36]:
# Obtener el optimizador del modelo
optimizer = modelLSTMFreeze.optimizer

# Obtener detalles del optimizador
print(f"Tipo de optimizador: {type(optimizer).__name__}")
print(f"Tasa de aprendizaje: {optimizer.learning_rate.numpy()}")
print(f"Epsilon: {optimizer.epsilon}")

Tipo de optimizador: RMSprop
Tasa de aprendizaje: 0.0010000000474974513
Epsilon: 1e-07


In [152]:
NUM_TRAINABLE = 1

numLSTM_layers = sum(1 for layer in modelLSTMFreeze.layers if "lstm" in layer.name)

numFreezeLayers = numLSTM_layers - NUM_TRAINABLE # Congelamos todas menos la útlima capa

numberLayersFreezed = 0

# Congelar las primeras 'numFreezeLayers' capas LSTM
for i, layer in enumerate(modelLSTMFreeze.layers):


    if "lstm" in layer.name and numFreezeLayers>numberLayersFreezed:
        numberLayersFreezed += 1
        layer.trainable = False

    print(layer, layer.trainable)

<LSTM name=lstm, built=True> False
<LSTM name=lstm_1, built=True> False
<LSTM name=lstm_2, built=True> False
<Dropout name=dropout_1, built=True> True
<LSTM name=lstm_3, built=True> True
<Dense name=dense_2, built=True> True
<Dropout name=dropout_2, built=True> True
<Dense name=dense_3, built=True> True


In [38]:
modelLSTMFreeze.compile(optimizer=optimizer, loss=MeanSquaredError(), metrics=[
                                MeanSquaredError(name='mse'),
                                RootMeanSquaredError(name='rmse'),
                                MeanAbsolutePercentageError(name='mape')
                            ])

In [41]:
# Comienza a medir el tiempo de entrenamiento
start_time = time.time()

historyLSTMTransfer = modelLSTMFreeze.fit(X_train, y_train, epochs=200, batch_size=24,
                        validation_data=(X_val, y_val), callbacks=[EarlyStopping(monitor='val_loss', patience=10)])

# Finaliza la medición del tiempo de entrenamiento
end_time = time.time()
print(f"[MODELO CONGELADO]: {end_time-start_time}")


Epoch 1/200
2360/2360 ━━━━━━━━━━━━━━━━━━━━ 16s 7ms/step - loss: 0.0146 - mape: 29.8034 - mse: 0.0146 - rmse: 0.1210 - val_loss: 0.0135 - val_mape: 24.7561 - val_mse: 0.0135 - val_rmse: 0.1162
Epoch 2/200
2360/2360 ━━━━━━━━━━━━━━━━━━━━ 17s 7ms/step - loss: 0.0146 - mape: 30.1204 - mse: 0.0146 - rmse: 0.1208 - val_loss: 0.0133 - val_mape: 25.3843 - val_mse: 0.0133 - val_rmse: 0.1154
Epoch 3/200
2360/2360 ━━━━━━━━━━━━━━━━━━━━ 17s 7ms/step - loss: 0.0144 - mape: 30.0842 - mse: 0.0144 - rmse: 0.1200 - val_loss: 0.0134 - val_mape: 25.2314 - val_mse: 0.0134 - val_rmse: 0.1157
Epoch 4/200
2360/2360 ━━━━━━━━━━━━━━━━━━━━ 13s 6ms/step - loss: 0.0145 - mape: 29.8935 - mse: 0.0145 - rmse: 0.1204 - val_loss: 0.0135 - val_mape: 27.2908 - val_mse: 0.0135 - val_rmse: 0.1162
Epoch 5/200
2360/2360 ━━━━━━━━━━━━━━━━━━━━ 14s 6ms/step - loss: 0.0146 - mape: 29.6769 - mse: 0.0146 - rmse: 0.1210 - val_loss: 0.0132 - val_mape: 26.2086 - val_mse: 0.0132 - val_rmse: 0.1151
Epoch 6/200
2360/2360 ━━━━━━━━━━━━━━━━━━

In [49]:
# CON EL MODELO PREENTRENADO NO HAY QUE HACER ESO!
# NO SE COMPILA NI NADA

###############################################################################
optimizerLSTM = modelLSTM.optimizer
print(optimizerLSTM)

modelLSTM.compile(optimizer=optimizer, loss=MeanSquaredError(), metrics=[
                                MeanSquaredError(name='mse'),
                                RootMeanSquaredError(name='rmse'),
                                MeanAbsolutePercentageError(name='mape')
                            ])
###############################################################################

In [52]:
# Comienza a medir el tiempo de entrenamiento
start_time = time.time()

historyLSTM = modelLSTM.fit(X_train, y_train, epochs=200, batch_size=24,
                    validation_data=(X_val, y_val), callbacks=[EarlyStopping(monitor='val_loss', patience=10)])

# Finaliza la medición del tiempo de entrenamiento
end_time = time.time()
print(f"[MODELO NUEVO]: {end_time-start_time}")

Epoch 1/200
2360/2360 ━━━━━━━━━━━━━━━━━━━━ 26s 10ms/step - loss: 0.0481 - mape: 95.6664 - mse: 0.0481 - rmse: 0.2193 - val_loss: 0.0484 - val_mape: 96.9497 - val_mse: 0.0484 - val_rmse: 0.2200
Epoch 2/200
2360/2360 ━━━━━━━━━━━━━━━━━━━━ 23s 10ms/step - loss: 0.0475 - mape: 89.8709 - mse: 0.0475 - rmse: 0.2180 - val_loss: 0.0465 - val_mape: 85.4891 - val_mse: 0.0465 - val_rmse: 0.2157
Epoch 3/200
2360/2360 ━━━━━━━━━━━━━━━━━━━━ 24s 10ms/step - loss: 0.0466 - mape: 89.9724 - mse: 0.0466 - rmse: 0.2158 - val_loss: 0.0466 - val_mape: 94.0187 - val_mse: 0.0466 - val_rmse: 0.2159
Epoch 4/200
2360/2360 ━━━━━━━━━━━━━━━━━━━━ 22s 9ms/step - loss: 0.0464 - mape: 87.8208 - mse: 0.0464 - rmse: 0.2153 - val_loss: 0.0540 - val_mape: 77.6771 - val_mse: 0.0540 - val_rmse: 0.2324
Epoch 5/200
2360/2360 ━━━━━━━━━━━━━━━━━━━━ 23s 10ms/step - loss: 0.0463 - mape: 85.2603 - mse: 0.0463 - rmse: 0.2152 - val_loss: 0.0455 - val_mape: 85.6459 - val_mse: 0.0455 - val_rmse: 0.2134
Epoch 6/200
2360/2360 ━━━━━━━━━━━━━━

In [32]:
# Realizar predicciones y calcular métricas para el conjunto de entrenamiento
predictions_train = predictionForIndividuals(X_train, y_train, modelLSTMFreeze, 24)
predictions_train["PredictionsDenormalize"] = predictions_train.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Predictions"), axis=1)
predictions_train["ActualDenormalize"] = predictions_train.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Actuals"), axis=1)

train_mse = mean_squared_error(predictions_train["ActualDenormalize"],predictions_train["PredictionsDenormalize"])
train_rmse = np.sqrt(train_mse)
train_mape = (np.sum(np.abs(predictions_train["PredictionsDenormalize"] - predictions_train["ActualDenormalize"])) / np.sum(np.abs(predictions_train["ActualDenormalize"]))) * 100
train_r2 = r2_score(predictions_train["ActualDenormalize"], predictions_train["PredictionsDenormalize"])

# Realizar predicciones y calcular métricas para el conjunto de validación
predictions_val = predictionForIndividuals(X_val, y_val, modelLSTMFreeze, 24)
predictions_val["PredictionsDenormalize"] = predictions_val.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Predictions"), axis=1)
predictions_val["ActualDenormalize"] = predictions_val.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Actuals"), axis=1)

val_mse = mean_squared_error(predictions_val["ActualDenormalize"],predictions_val["PredictionsDenormalize"])
val_rmse = np.sqrt(val_mse)
val_mape = (np.sum(np.abs(predictions_val["PredictionsDenormalize"] - predictions_val["ActualDenormalize"])) / np.sum(np.abs(predictions_val["ActualDenormalize"]))) * 100
val_r2 = r2_score(predictions_val["ActualDenormalize"], predictions_val["PredictionsDenormalize"])

# Realizar predicciones y calcular métricas para el conjunto de prueba
predictions_test = predictionForIndividuals(X_test, y_test, modelLSTMFreeze, 24)
predictions_test["PredictionsDenormalize"] = predictions_test.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Predictions"), axis=1)
predictions_test["ActualDenormalize"] = predictions_test.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Actuals"), axis=1)

test_mse = mean_squared_error(predictions_test["ActualDenormalize"],predictions_test["PredictionsDenormalize"])
test_rmse = np.sqrt(test_mse)
test_mape = (np.sum(np.abs(predictions_test["PredictionsDenormalize"] - predictions_test["ActualDenormalize"])) / np.sum(np.abs(predictions_test["ActualDenormalize"]))) * 100
test_r2 = r2_score(predictions_test["ActualDenormalize"], predictions_test["PredictionsDenormalize"])

print(f"RESULTADOS DE MSE, RMSE, R2, MAPE (Train): {train_mse}, {train_rmse}, {train_r2}, {train_mape}")
print(f"RESULTADOS DE MSE, RMSE, R2, MAPE (Val): {val_mse}, {val_rmse}, {val_r2}, {val_mape}")
print(f"RESULTADOS DE MSE, RMSE, R2, MAPE (Test): {test_mse}, {test_rmse}, {test_r2}, {test_mape}")

/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:191: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictionsDF = pd.concat([predictionsDF, tempDF], ignore_index=True)
/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:191: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictionsDF = pd.concat([predictionsDF, tempDF], ignore_index=True)
/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:191: FutureWarning: The behavior of Dat

RESULTADOS DE MSE, RMSE, R2, MAPE (Train): 17.993569741393436, 4.241882806183291, 0.842134186001047, 20.555789188600222
RESULTADOS DE MSE, RMSE, R2, MAPE (Val): 18.37357205125532, 4.286440487310575, 0.8325865317341601, 21.15726288156048
RESULTADOS DE MSE, RMSE, R2, MAPE (Test): 19.06853804560989, 4.3667537193675, 0.8585828415279593, 20.61283012417917


In [154]:
# Creamos un df que contiene los resultados de los modelos freeze de cada especie
df_Freeze = pd.DataFrame(columns = ["File", "Time", "TrainMSE", "TrainRMSE", "TrainR2", "TrainMAPE", 
                                    "ValidationMSE", "ValidationRMSE", "ValidationR2", "ValidationMAPE",
                                    "TestMSE", "TestRMSE", "TestR2", "TestMAPE"])

for archivo in os.listdir("RCPMerged"):

    # Cargar el archivo CSV con punto y coma como delimitador
    df = pd.read_csv(f'RCPMerged/{archivo}')
    df = df[~df["nametag"].str.startswith("INDI005")]

    # Codificamos, normalización y split de datos
    df = codification(df)
    print(df.shape)

    df, valorNormalizacion = individualNormalization(df)
    print(f"SE HA NORMALIZADO EL ARCHIVO: {archivo}")

    temp_df = df
    train_data, val_data, test_data = split_population_individuals(temp_df, train_pct=0.80, val_pct_in_train=0.20, details=False)
    train_data.shape, val_data.shape, test_data.shape

    # Obtenemos X e y para los datasets de train, val y test 
    WINDOWS_SIZE = 3
    X_train, y_train = df_to_X_y_ind_3(train_data, WINDOWS_SIZE)
    X_val, y_val = df_to_X_y_ind_3(val_data, WINDOWS_SIZE)
    X_test, y_test = df_to_X_y_ind_3(test_data, WINDOWS_SIZE)
    print(X_train.shape, X_test.shape, X_val.shape)

    # Cargamos el modelo global
    modelLSTM = tf.keras.models.load_model("models/modelLSTM.keras")
    modelLSTMFreeze = modelLSTM

    # Obtener el optimizador del modelo
    optimizer = modelLSTMFreeze.optimizer

    # Indicamos el numero de layers a entrenar
    NUM_TRAINABLE = 1

    numLSTM_layers = sum(1 for layer in modelLSTMFreeze.layers if "lstm" in layer.name)

    numFreezeLayers = numLSTM_layers - NUM_TRAINABLE # Congelamos todas menos la útlima capa

    numberLayersFreezed = 0

    # Congelar las primeras 'numFreezeLayers' capas LSTM
    for i, layer in enumerate(modelLSTMFreeze.layers):

        if "lstm" in layer.name and numFreezeLayers>numberLayersFreezed:
            numberLayersFreezed += 1
            layer.trainable = False

        print(layer, layer.trainable)

    # Compilamos el modelo con los nuevos datos
    modelLSTMFreeze.compile(optimizer=optimizer, loss=MeanSquaredError(), metrics=[
                                MeanSquaredError(name='mse'),
                                RootMeanSquaredError(name='rmse'),
                                MeanAbsolutePercentageError(name='mape')
                            ])
    
    # Comienza a medir el tiempo de entrenamiento
    start_time = time.time()

    historyLSTMTransfer = modelLSTMFreeze.fit(X_train, y_train, epochs=200, batch_size=24,
                            validation_data=(X_val, y_val), callbacks=[EarlyStopping(monitor='val_loss', patience=10)], verbose=0)

    # Finaliza la medición del tiempo de entrenamiento
    end_time = time.time()
    print(f"[MODELO CONGELADO]: {end_time-start_time}")

    # Realizar predicciones y calcular métricas para el conjunto de entrenamiento
    predictions_train = predictionForIndividuals(X_train, y_train, modelLSTMFreeze, 24)
    predictions_train["PredictionsDenormalize"] = predictions_train.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Predictions"), axis=1)
    predictions_train["ActualDenormalize"] = predictions_train.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Actuals"), axis=1)

    train_mse = mean_squared_error(predictions_train["ActualDenormalize"],predictions_train["PredictionsDenormalize"])
    train_rmse = np.sqrt(train_mse)
    train_mape = (np.sum(np.abs(predictions_train["PredictionsDenormalize"] - predictions_train["ActualDenormalize"])) / np.sum(np.abs(predictions_train["ActualDenormalize"]))) * 100
    train_r2 = r2_score(predictions_train["ActualDenormalize"], predictions_train["PredictionsDenormalize"])

    # Realizar predicciones y calcular métricas para el conjunto de validación
    predictions_val = predictionForIndividuals(X_val, y_val, modelLSTMFreeze, 24)
    predictions_val["PredictionsDenormalize"] = predictions_val.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Predictions"), axis=1)
    predictions_val["ActualDenormalize"] = predictions_val.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Actuals"), axis=1)

    val_mse = mean_squared_error(predictions_val["ActualDenormalize"],predictions_val["PredictionsDenormalize"])
    val_rmse = np.sqrt(val_mse)
    val_mape = (np.sum(np.abs(predictions_val["PredictionsDenormalize"] - predictions_val["ActualDenormalize"])) / np.sum(np.abs(predictions_val["ActualDenormalize"]))) * 100
    val_r2 = r2_score(predictions_val["ActualDenormalize"], predictions_val["PredictionsDenormalize"])

    # Realizar predicciones y calcular métricas para el conjunto de prueba
    predictions_test = predictionForIndividuals(X_test, y_test, modelLSTMFreeze, 24)
    predictions_test["PredictionsDenormalize"] = predictions_test.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Predictions"), axis=1)
    predictions_test["ActualDenormalize"] = predictions_test.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Actuals"), axis=1)

    test_mse = mean_squared_error(predictions_test["ActualDenormalize"],predictions_test["PredictionsDenormalize"])
    test_rmse = np.sqrt(test_mse)
    test_mape = (np.sum(np.abs(predictions_test["PredictionsDenormalize"] - predictions_test["ActualDenormalize"])) / np.sum(np.abs(predictions_test["ActualDenormalize"]))) * 100
    test_r2 = r2_score(predictions_test["ActualDenormalize"], predictions_test["PredictionsDenormalize"])

    print(f"RESULTADOS DE MSE, RMSE, R2, MAPE (Train): {train_mse}, {train_rmse}, {train_r2}, {train_mape}")
    print(f"RESULTADOS DE MSE, RMSE, R2, MAPE (Val): {val_mse}, {val_rmse}, {val_r2}, {val_mape}")
    print(f"RESULTADOS DE MSE, RMSE, R2, MAPE (Test): {test_mse}, {test_rmse}, {test_r2}, {test_mape}")

    # Guardamos los datos calculados
    df_Freeze.loc[len(df_Freeze)] = [archivo, end_time-start_time,train_mse, train_rmse, train_r2, train_mape, val_mse, val_rmse, val_r2, val_mape, test_mse, test_rmse, test_r2, test_mape]

df_Freeze

(96429, 77)
SE HA NORMALIZADO EL ARCHIVO: Abies spectabilis_merged.csv
(56618, 4, 43) (19637, 4, 43) (15689, 4, 43)
<LSTM name=lstm, built=True> False
<LSTM name=lstm_1, built=True> False
<LSTM name=lstm_2, built=True> False
<Dropout name=dropout_1, built=True> True
<LSTM name=lstm_3, built=True> True
<Dense name=dense_2, built=True> True
<Dropout name=dropout_2, built=True> True
<Dense name=dense_3, built=True> True


/home/workstation/Escritorio/PabloC/TransferLearning/.venv/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 18 variables whereas the saved optimizer has 34 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


[MODELO CONGELADO]: 629.6567380428314


/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:191: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictionsDF = pd.concat([predictionsDF, tempDF], ignore_index=True)
/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:191: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictionsDF = pd.concat([predictionsDF, tempDF], ignore_index=True)
/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:191: FutureWarning: The behavior of Dat

RESULTADOS DE MSE, RMSE, R2, MAPE (Train): 18.05214589999953, 4.248781695968802, 0.8416202705805838, 20.77368072262704
RESULTADOS DE MSE, RMSE, R2, MAPE (Val): 18.66985577891557, 4.320862851204094, 0.8298868995505018, 21.598038885740245
RESULTADOS DE MSE, RMSE, R2, MAPE (Test): 19.006251421575207, 4.359615971800178, 0.8590447750731913, 20.799570184748596
(34476, 77)
SE HA NORMALIZADO EL ARCHIVO: Picea smithiana_merged.csv
(19933, 4, 43) (7272, 4, 43) (5863, 4, 43)
<LSTM name=lstm, built=True> False
<LSTM name=lstm_1, built=True> False
<LSTM name=lstm_2, built=True> False
<Dropout name=dropout_1, built=True> True
<LSTM name=lstm_3, built=True> True
<Dense name=dense_2, built=True> True
<Dropout name=dropout_2, built=True> True
<Dense name=dense_3, built=True> True


...

/home/workstation/Escritorio/PabloC/TransferLearning/.venv/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 18 variables whereas the saved optimizer has 34 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


[MODELO CONGELADO]: 137.7366373538971


/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:191: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictionsDF = pd.concat([predictionsDF, tempDF], ignore_index=True)
/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:191: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictionsDF = pd.concat([predictionsDF, tempDF], ignore_index=True)
/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:191: FutureWarning: The behavior of Dat

RESULTADOS DE MSE, RMSE, R2, MAPE (Train): 7.866423968169519, 2.804714596562281, 0.8717040148129338, 21.799637906897935
RESULTADOS DE MSE, RMSE, R2, MAPE (Val): 3.158001087312235, 1.7770765564016184, 0.8394732392792952, 23.936873677033812
RESULTADOS DE MSE, RMSE, R2, MAPE (Test): 10.336970617061363, 3.2151159570163816, 0.9010781398681826, 21.59677287568315


,File,Time,TrainMSE,TrainRMSE,TrainR2,TrainMAPE,ValidationMSE,ValidationRMSE,ValidationR2,ValidationMAPE,TestMSE,TestRMSE,TestR2,TestMAPE
0,Abies spectabilis_merged.csv,629.656738,18.052146,4.248782,0.841620,20.773681,18.669856,4.320863,0.829887,21.598039,19.006251,4.359616,0.859045,20.799570
1,Picea smithiana_merged.csv,532.864744,55.249016,7.432968,0.838089,21.703452,60.392050,7.771232,0.825725,22.599518,57.548815,7.586094,0.805660,23.605684
2,Abies pindrow_merged.csv,94.485225,46.531370,6.821391,0.800272,20.916621,38.119413,6.174092,0.782548,19.580132,52.524785,7.247398,0.628665,20.890326
3,Juniperus excelsa M.-Bieb_merged.csv,70.077633,5.591929,2.364726,0.772699,23.860241,6.283398,2.506671,0.760375,25.750070,12.556333,3.543492,0.813842,25.012970
4,Juniperus turkestanica Komar._merged.csv,26.968911,3.214600,1.792930,0.928384,18.119509,2.185921,1.478486,0.872883,19.935133,1.094899,1.046374,0.798569,20.710972
5,Pinus roxburghii_merged.csv,128.075591,50.853994,7.131199,0.809347,21.873852,39.677191,6.298983,0.822876,22.564124,57.249992,7.566372,0.759767,24.280650
6,Pinus wallichiana_merged.csv,137.781056,9.157709,3.026171,0.864153,16.645561,8.543942,2.923002,0.857085,17.331696,12.629798,3.553843,0.700458,22.451586
7,Pinus gerardiana_merged.csv,132.807315,10.136212,3.183742,0.738873,24.759584,12.281940,3.504560,0.689615,24.929726,9.486489,3.080014,0.763873,23.556891
8,Populus ciliata_merged.csv,26.633441,100.995015,10.049628,0.721925,26.013534,128.027944,11.314943,0.454499,34.588940,70.928948,8.421933,0.645585,30.425175
9,Betula utilis_merged.csv,54.628330,1.538910,1.240528,0.772818,25.119829,0.392305,0.626343,0.795701,22.244111,0.383480,0.619258,0.853453,22.327224


In [156]:
df_Freeze.to_csv("resultadosTransfer.csv", index=False)